# Symbolic Differentiation 

In this notebook our goal is to implement *symbolic differentiation*.  Concretely, we want do implement a function `diff` that takes one argument:
  - The argument `expr` represents an *arithmetic expression*.
    Here an arithmetic expression is any string that is build from variable and numbers by application
    of any of the operator symbols "`+`", "`-`", "`*`", "`/`", and "`**`".
    The operator "`**`" represents exponentiation, i.e. an expression of the form 
    $a \texttt{**} b$ is interpreted as $a^b$.          
    Furthermore, if $e$ is an expression, then both $\exp(e)$ and $\ln(e)$ are expressions too.

The function call `diff(expr)` will then compute the derivative of `expr` with respect to the variable `x`.  For example, the function call 
`diff("x * exp(x)")` will compute the output
`1 * exp(x) + x * exp(x)` because we have:
$$ \frac{\mathrm{d}\;}{\mathrm{d}x} \bigl( x \cdot \mathrm{e}^x \bigr) = 1 \cdot \mathrm{e}^x + x \cdot \mathrm{e}^x. $$

This file shows the implementation of a program that can perform *symbolic differentiation* using `Ply`.  The grammar for the language implemented by this parser is as follows:
$$
\begin{array}{lcl}
  \texttt{expr}    & \rightarrow & \;\texttt{expr}\; \texttt{'+'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{expr}\; \texttt{'-'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{product}                                  \\[0.2cm]
  \texttt{product} & \rightarrow & \;\texttt{product}\; \texttt{'*'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{product}\; \texttt{'/'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{factor}                                   \\[0.2cm]
  \texttt{factor}  & \rightarrow & \texttt{base} \;\texttt{'**'} \; \texttt{factor}    \\
                   & \mid        & \texttt{base}                                       \\[0.2cm]
  \texttt{base}    & \rightarrow & \texttt{exp} \; \texttt{'('} \; \texttt{expr} \;\texttt{')'}     \\
                   & \mid        & \texttt{ln} \; \texttt{'('} \; \texttt{expr} \;\texttt{')'}      \\
                   & \mid        & \texttt{'('} \; \texttt{expr} \;\texttt{')'}                     \\
                   & \mid        & \;\texttt{NUMBER}                                                \\
                   & \mid        & \;\texttt{'x'}                               
\end{array}
$$

## Specification of the Scanner

In [1]:
import ply.lex as lex

Most of the tokens consist only of a single character and can therefore be defined as *literals*.
Hence, we only have to define four tokens explicitely.

In [2]:
tokens = [ 'NUMBER', 'EXP', 'LN', 'POWER' ]

The token `NUMBER` specifies a *natural number*.

In [3]:
def t_NUMBER(t):
    r'0|[1-9][0-9]*'
    t.value = int(t.value)
    return t

In [4]:
t_EXP = r'exp'

In [5]:
t_LN = r'ln'

Below, we need to escape the meta charater `*`.

In [6]:
t_POWER = r'\*\*'

In [7]:
literals = ['+', '-', '*', '/', '(', ')', 'x']

Blanks and tabulators are ignored.

In [8]:
t_ignore  = ' \t'

Unkown characters are reported as lexical errors.

In [9]:
def t_error(t):
    print(f"Illegal character '{t.value[0]}' at character number {t.lexer.lexpos}.")
    t.lexer.skip(1)

The next line is necessary because we use `Ply` from a notebook.

In [10]:
__file__ = 'main'

We generate the lexer.

In [11]:
lexer = lex.lex()

## Specification of the Parser

In [12]:
import ply.yacc as yacc

The *start variable* of our grammar is `statement`.

In [13]:
start = 'expr'

An *expr* is a sequence of *products* that are combined with the operator `+`.
The corresponding grammar rules are:
```
    expr : expr '+' product
         | expr '-' product
         | product
         ;
```

In [14]:
def p_expr_plus(p):
    "expr : expr '+' product"
    p[0] = ('+', p[1], p[3])

def p_expr_minus(p):
    "expr : expr '-' product"
    p[0] = ('-', p[1], p[3])
    
def p_expr_product(p):
    "expr : product"
    p[0] = p[1]

A *product* is a sequence of *factors* that are combined with the operator `*`.
The corresponding grammar rules are:
```
    product : product '*' factor
            | product '/' factor
            | factor
            ;
```

In [15]:
def p_product_multiply(p):
    "product : product '*' factor"
    p[0] = ('*', p[1], p[3])

def p_product_divide(p):
    "product : product '/' factor"
    p[0] = ('/', p[1], p[3])
    
def p_product_factor(p):
    "product : factor"
    p[0] = p[1]

A `factor` is either a `base` expression raised to a power or it is just a `base` expression.
```
   factor : base '**' factor
          | base
          ;
```

In [16]:
def p_factor_power(p):
    "factor : base POWER factor"
    p[0] = ('**', p[1], p[3])

def p_factor_base(p):
    "factor : base"
    p[0] = p[1]

A `base` expression can be the application of a function, an expression in parenthesis, a number, or the variable `x`.
```
    base : exp '(' expr ')'
         | ln  '(' expr ')'
         | '(' expr ')'
         | NUMBER
         | 'x'
         ;
```

In [17]:
def p_base_exp(p):
    "base : EXP '(' expr ')'"
    p[0] = ('exp', p[3])

def p_base_ln(p):
    "base : LN '(' expr ')'"
    p[0] = ('ln', p[3])
    
def p_base_group(p):
    "base : '(' expr ')'"
    p[0] = p[2]

def p_base_number(p):
    "base : NUMBER"
    p[0] = p[1]

def p_base_x(p):
    "base : 'x'"
    p[0] = p[1]

The method `p_error` is called if a syntax error occurs.  The argument `p` is the token that could not be read.  If `p` is `None` then the syntax error occurs at the end of the input.

In [18]:
def p_error(p):
    if p:
        print(f"Syntax error at character number {p.lexer.lexpos} at token '{p.value}'.")
    else:
        raise(f"Incomplete input.")

In [19]:
parser = yacc.yacc(write_tables=False, debug=True)

Generating LALR tables


The parser is invoked by calling the method `yacc.parse(s)` where `s` is a string that is to be parsed.

In [20]:
def parse(s):
    return yacc.parse(s)

In [21]:
parse('ln(x ** x) + exp(x * x)')

('+', ('ln', ('**', 'x', 'x')), ('exp', ('*', 'x', 'x')))

Now we are ready to implement the function `diff` that takes an expression possibly containing the variable `x`.
It computes the derivative of the given expression with respect to `x`.
<ol>
<li>  The lines 4 - 6 implement the rule: 
      $$\frac{\mathrm{d}\;}{\mathrm{d}x}\bigl(f(x) + g(x)\bigr) = \frac{\mathrm{d}\;}{\mathrm{d}x} f(x) + \frac{\mathrm{d}\;}{\mathrm{d}x} g(x)$$
      </li>
<li>  Line 7 - 9 implement the rule:
      $$\frac{\mathrm{d}\;}{\mathrm{d}x}\bigl(f(x) - g(x)\bigr) = \frac{\mathrm{d}\;}{\mathrm{d}x} f(x) - \frac{\mathrm{d}\;}{\mathrm{d}x} g(x)$$
      </li>
<li>  Line 10 - 12 deals with the case where <tt>e</tt> is a product.  The 
      <a href="https://en.wikipedia.org/wiki/Product_rule">product rule</a> is      
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x}\bigl(f(x) \cdot g(x)\bigr) = \left(\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)\right)\cdot g(x) + f(x) \cdot \left(\frac{\mathrm{d}\;}{\mathrm{d}x} g(x)\right)
      $$
      </li>
<li>  Line 13 - 15 deals with the case where <tt>e</tt> is a quotient.  The
      <a href="https://en.wikipedia.org/wiki/Quotient_rule">quotient rule</a> is
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x}\left(\frac{f(x)}{g(x)}\right) = 
         \frac{\displaystyle\left(\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)\right)\cdot g(x) - 
         f(x) \cdot \left(\frac{\mathrm{d}\;}{\mathrm{d}x} g(x)\right)}{g(x) \cdot g(x)}
      $$
      </li>
<li>  Line 16 - 18 deals with the case where <tt>e</tt> is a power.  Now in order to take the derivative of an
      expression of the form
      $$  f(x)^{g(x)} $$
      we first need to rewrite this expression using the following trick:
      $$ f(x)^{g(x)} = \exp\bigl(\ln\bigl(f(x)^{g(x)}\bigr)\bigr) = \exp\bigl(g(x) \cdot \ln(f(x))\bigr) $$
      Then, we can recursively call <tt>diff</tt> for this expression.  This works, because the function
      <tt>diff</tt> can deal with both the exponential function $x \mapsto \exp(x)$ and with the natural
      logarithm $x \mapsto \ln(x)$.  This rewriting is done in line 21.
      </li>
<li>  Line 19-21 deals with the case where <tt>e</tt> has the form 
      $$\ln\bigl(f(x)\bigr)$$  
      In order to take the derivative of this expression, we first need to know the derivative of the natural
      logarithm.  This derivative is given as     
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x} \ln(x) = \frac{1}{x}$$
      Then, using the <a href="https://en.wikipedia.org/wiki/Chain_rule">chain rule</a> we have that
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x} \ln\bigl(f(x)\bigr) = \frac{\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)}{f(x)}$$
      </li>
<li>  Line 22 - 24 deals with the case where <tt>e</tt> has the form $\exp\bigl(f(x)\bigr)$.  
      In order to take the derivative of this expression, we first need to know the derivative of the 
      <a href="https://en.wikipedia.org/wiki/Exponential_function">exponential function</a>.  
      This derivative is given as 
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x} \exp(x) = \exp(x)$$    
      Then, using the <a href="https://en.wikipedia.org/wiki/Chain_rule">chain rule</a> we have that
      $$\frac{\mathrm{d}\;}{\mathrm{d}x} \exp\bigl(f(x)\bigr) = \left(\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)\right) \cdot \exp\bigl(f(x)\bigr)
      $$
      </li>
<li>  Line 25-26 deals with the case where <tt>e</tt> is a variable and happens to be the same variable as
      <tt>x</tt>.   As we have
      $$\frac{\mathrm{d}x}{\mathrm{d}x} = 1,$$
      the function <tt>diff</tt> returns <tt>1</tt> in this case.
      </li>
<li>  Otherwise, the expression is assumed to be a constant and hence we return 0.
      </li>
</ol>


In [22]:
def diff(e):
    "differentiate the expression e with respect to the variable x"
    match e:
        case ('+', f, g):
            fs, gs = diff(f), diff(g)
            return ('+', fs, gs)
        case ('-', f, g):
            fs, gs = diff(f), diff(g)
            return ('-', fs, gs)
        case ('*', f, g):
            fs, gs = diff(f), diff(g)
            return ('+', ('*', fs, g), ('*', f, gs))
        case ('/', f, g):
            fs, gs = diff(f), diff(g)
            return ('/', ('-', ('*', fs, g), ('*', f, gs)), ('*', g, g))
        case ('**', f, g):
            return diff(('exp', ('*', g, ('ln', f))))
        case ('ln', f):
            fs = diff(f) 
            return ('/', fs, f)
        case ('exp', f):
            fs = diff(f)
            return ('*', fs, e)
        case 'x':
            return 1
    return 0

The function below turns a nested tuple representing a function into a readable string.

In [23]:
def toString(e):
    if isinstance(e, (int, str)):
        return str(e)
    if len(e) == 2:
        return e[0] + '(' + toString(e[1]) + ')'
    if e[0] == '+':
        return toString(e[1]) + ' + ' + toString(e[2])
    if e[0] == '-':
        lhs = toString(e[1])
        if precedenceOp(e[2]) == 1:
            rhs = '(' + toString(e[2]) + ')'
        else:
            rhs = toString(e[2])
        return lhs + ' - ' + rhs
    if e[0] == '*':
        if precedenceOp(e[1]) == 1:
           lhs = '(' + toString(e[1]) + ')'
        else:
           lhs = toString(e[1])
        if precedenceOp(e[2]) == 1:
            rhs = '(' + toString(e[2]) + ')'
        else:
            rhs = toString(e[2])
        return lhs + '*' + rhs
    if e[0] == '/':
        if precedenceOp(e[1]) == 1:
            lhs = '(' + toString(e[1]) + ')'
        else:
            lhs = toString(e[1])
        if precedenceOp(e[2]) <= 2:
            rhs = '(' + toString(e[2]) + ')'
        else:
            rhs = toString(e[2])
        return lhs + '/' + rhs
    if e[0] == '**':
        if precedenceOp(e[1]) <= 3:
            lhs = '(' + toString(e[1]) + ')'
        else:
            lhs = toString(e[1])
        if precedenceOp(e[2]) <= 2:
            rhs = '(' + toString(e[2]) + ')'
        else:
            rhs = toString(e[2])
        return lhs + '**' + rhs

def precedenceOp(expr):
    if isinstance(expr, tuple):
        return precedence(expr[0])
    return 4

def precedence(op: str):
    Precedences = { '+': 1, '-': 1, '*': 2, '/': 2, '**': 3 }
    if op in Precedences:
        return Precedences[op]
    return 4

In [24]:
def test(s):
    t = parse(s)
    d = diff(t)
    print(f"d/dx {s} = {toString(d)}")

In [25]:
test("x ** x")

d/dx x ** x = (1*ln(x) + x*1/x)*exp(x*ln(x))


In [26]:
test("x * ln(x) / exp(x/x)")

d/dx x * ln(x) / exp(x/x) = ((1*ln(x) + x*1/x)*exp(x/x) - x*ln(x)*(1*x - x*1)/(x*x)*exp(x/x))/(exp(x/x)*exp(x/x))
